# Overview
This notebook walks through the current RAG retrieval method used on an Obsidian notes directory containing knowledge on growing Cannabis in Living Soil.



# Load the Markdown Files
Using Obsidian notes provides several advantages:
- Notes are easy to write and edit.
- Frontmatter/tags transfer into the metadata of the nodes.
- The Headers provide natural splitting points for the text.

The `IngestService` class relies on Langchain's `ObsidianLoader` class to load the notes.  The `ObsidianLoader` class uses the frontmatter, tags, dataview fields, and file metadata to populate the metadata of the nodes.  The additional metadata will benefit retrieval of the best nodes to answer a given query.

The `load_obsidian_notes` method takes in either:
- a list of strings where each string is considered a markdown file.
- a directory path to an Obsidian vault.
The list of strings options is useful for testing.

In [5]:
# Example of an obsidian note:
doc = """#Calcium_additive #raise_ph #Wollastonite #Silicon_additive #buffer_pH #Calcium
Growers  turn to Wollastonite for:
- Its **liming** capability.  Wollastonite's dissolution rate is slower than agricultural lime, offering a buffering effect against rapid pH changes. This makes Wollastonite beneficial in areas with fluctuating acidity levels.
- Adding **Silicon**.
- Adding **Calcium**.
Wollastonite's pH buffering effect and Silicon content contribute to pest control and powdery mildew suppression, although the exact mechanisms are not fully understood.

# What is Wollastonite?

## Formation
Wollastonite is formed when Limestone is subjected to heat and pressure during metamorphism if surrounding silicate minerals are present.
### Basic Reaction:
Given high pressure and high temperature:
- CaCO3 (Limestone) + SiO2 (silica) → CaSiO3 (Wollastonite) + CO2 (carbon Dioxide)
## Sources
China is the largest producer of Wollastonite. Other areas where Wollastonite is mined include the United States (although it was originally mined in California, the only active mining in the U.S. is now in New York State), India, Mexico, Canada, and Finland.

## Industrial Applications of Wollastonite

|Industry|Application|
|---|---|
|Ceramics|Smoother and more durable ceramics, reinforcement agent|
|Plastics and Rubber|Cost-effective strengthening agent|
|Paints and Coatings|Reinforcement, improved durability and impact resistance|
|Construction|Improved strength and durability of building materials, safe alternative to asbestos|
##  How Wollastonite Provides Plants with Ca and Si

Wollastonite reacts with Water and Carbon Dioxide in the soil to form Calcium Bicarbonate and Silicon Dioxide.
- CaSiO₃ (Wollastonite)+2CO₂ (carbon Dioxide,)+H₂O (Water)→Ca(HCO₃)₂ (Calcium bicarbonate)+SiO₂ (silica)

### Calcium
- Calcium bicarbonate  (Ca(HCO₃)₂) is unstable and fairly easily decomposes to Limestone (CaCO₃):
		- Ca(HCO₃)₂ (Calcium bicarbonate)→CaCO₃ (Limestone)+  CO₂ (carbon Dioxide) + H₂O (Water)

- Soils with a pH below 7 (acidic soils) contain hydrogen ions (H+). These hydrogen ions react with the Limestone (CaCO3) to form Calcium ions (Ca2+), Water (H2O), and Carbon Dioxide (CO2).
	- CaCO3 (Limestone) + 2H+ (hydrogen ions) → Ca2+ (Calcium ions) + H2O (Water) + CO2 (carbon Dioxide)
### Silicon
- Silicon Dioxide slowly breaks down into Silicic Acid, which plants absorb. This process is influenced by soil pH, temperature, and microbial activity.
	- SiO2 (Silicon Dioxide) + 2H2O (Water) → H4SiO4 (Silicic Acid)

- Plants absorb Silicic Acid from the soil solution through their roots.


"""

In [10]:
# --->: Read in the markdown files in the Obsidian vault directory
from src.ingest_service import IngestService
# The Directory containing the knowledge documents used by the AI to do the analysis on the soil tests.
# soil_knowledge_directory = r"G:\My Drive\Audios_To_Knowledge\knowledge\AskGrowBuddy\AskGrowBuddy\Knowledge\soil_test_knowlege"
# Load the documents
ingest_service = IngestService()
# loaded_documents = ingest_service.load_obsidian_notes(soil_knowledge_directory)
loaded_documents = ingest_service.load_obsidian_notes([doc])
# Show some summary stats about the documents
from src.doc_stats import DocStats
DocStats.print_llama_index_docs_summary_stats(loaded_documents)

ModuleNotFoundError: No module named 'src'

# Split the Obsidian notes into chunks
LlamaIndex's `MarkdownNodeParser` class is used to split the documents into chunks.  This allows for natural splitting of the text using the headers in the markdown files.     


In [ ]:
# --->: Chunk the documents
# LlamaIndex likes to call these nodes.
# limited_docs = loaded_documents[:2]
limited_docs = loaded_documents
nodes = ingest_service.chunk_text(limited_docs)
print(f"Number of documents used for chunking: {len(limited_docs)}")
print(f"Number of nodes created: {len(nodes)}")

# Set up Ollama
We are using Ollama to provide both the embedding model and the LLM.

In [1]:
# --->: Set up the local embedding model and LLM
# Set embedding model
from llama_index.core import Settings
from llama_index.llms.ollama import Ollama
from llama_index.embeddings.ollama import OllamaEmbedding

Settings.embed_model = OllamaEmbedding(
    model_name='nomic-embed-text',
    base_url="http://localhost:11434",
    ollama_additional_kwargs={"mirostat": 0},
)
# Choose your LLM...
Settings.llm = Ollama(model='mistral', request_timeout=1000.0)


# Set Up Multi-Index Fusion Retrieval
RAG will use three index/retrieval methods:
1. Vector Store Index. This provides a similarity search on vector embeddings of the nodes.
2. BM25 Index. This offers keyword-based retrieval, ranking documents based on term frequency and inverse document frequency.
3. Knowledge Graph Index. This captures relationships between entities and concepts, enabling context-aware and relationship-based retrieval.

The retrieved nodes are then passed through a Cohere's Reranker to rerank the nodes.  

By using a combination of these methods, we create a retrieval system that integrates semantic similarity, keyword relevance, and relational context. This approach aims to improve the likelihood of retrieving relevant information compared to using vector similarity search alone.


# Create and Persist the Vector Store Index
Chroma is used to store and query the vector embeddings of the nodes.  Ollama embeddings are used to vectorize the text.
## Simple Testing       
For simple testing, the overhead of using a Chroma db can be avoided.  This code proved useful:
```
from llama_index.core import VectorStoreIndex
vector_index = VectorStoreIndex(nodes)
cache_dir = "./vector_index_cache"
vector_index.storage_context.persist(persist_dir=cache_dir)
```

In [ ]:

ingest_service = IngestService()
# Create a Chroma collection object of a given name. Metadata, embeddings, text are all added.
our_collection = ingest_service.create_collection(docs=nodes, collection='test', embedding_model='nomic-embed-text')


In [ ]:
from llama_index.core import VectorStoreIndex
from llama_index.vector_stores.chroma import ChromaVectorStore
from src.ingest_service import IngestService
# Grab the vector index
ingest_service = IngestService()
our_collection = ingest_service.get_collection('test')
chroma_vector_store = ChromaVectorStore(chroma_collection=our_collection)
# Create a VectorStoreIndex using the ChromaVectorStore
vector_index = VectorStoreIndex.from_vector_store(chroma_vector_store)

# Create and Persist a Knowledge Graph Index


In [2]:
import nest_asyncio
nest_asyncio.apply()

In [4]:
from llama_index.graph_stores.neo4j import Neo4jPropertyGraphStore
neo4j_uri = "bolt://localhost:7687"  # Update with your Neo4j instance details
neo4j_user = "neo4j"
neo4j_password = "asd@123qwe"
graph_store = Neo4jPropertyGraphStore(
    username=neo4j_user,
    password=neo4j_password,
    url=neo4j_uri,
    database="test",  # Use appropriate database name
)

Received notification from DBMS server: {severity: WARNING} {code: Neo.ClientNotification.Statement.FeatureDeprecationWarning} {category: DEPRECATION} {title: This feature is deprecated and will be removed in future versions.} {description: The procedure has a deprecated field. ('config' used by 'apoc.meta.graphSample' is deprecated.)} {position: line: 1, column: 1, offset: 0} for query: "CALL apoc.meta.graphSample() YIELD nodes, relationships RETURN nodes, [rel in relationships | {name:apoc.any.property(rel, 'type'), count: apoc.any.property(rel, 'count')}] AS relationships"


In [ ]:
kg_index = ProeprtyGraphIndex.from_documents(nodes,embed_model=Settings.embed_model, kg_extractors=[SchemaLLMPathExtractor(llm=Settings.llm, temperature=0.01)],property_graph=graph_store,
show_progress=True)

In [ ]:
from llama_index.core import  KnowledgeGraphIndex
kg_index = KnowledgeGraphIndex(nodes, graph_store=graph_store,max_triplets_per_chunk=2,show_progress=True)

In [ ]:
   for node_id, node in kg_index.docstore.docs.items():
       print(f"Node ID: {node_id}")
       print(f"Node content: {node.get_content()}")
       print(f"Node metadata: {node.metadata}")
       print("---")

In [ ]:
from llama_index.core import  PropertyGraphIndex
from llama_index.graph_stores.neo4j import Neo4jPropertyGraphStore
# Define Neo4j connection credentials
neo4j_uri = "bolt://localhost:7687"  # Update with your Neo4j instance details
neo4j_user = "neo4j"
neo4j_password = "asd@123qwe"
# neo4j_password = os.getenv("NEO4J_PASSWORD")  # Securely use an environment variable

# Instantiate Neo4jGraphStore
graph_store = Neo4jPropertyGraphStore(
    username=neo4j_user,
    password=neo4j_password,
    url=neo4j_uri,
    database="test",  # Use appropriate database name
)

# According to LlamaIndex, I should be using PropertyGraphIndex instead of KnowledgeGraphIndex. However, it is not possible
# because PropertyGraphIndex throws the exception: AttributeError: 'Neo4jGraphStore' object has no attribute 'supports_vector_queries'
# Create Knowledge Graph Index from documents
# from llama_index.core import StorageContext
# storage_context = StorageContext.from_defaults(graph_store=graph_store)
from llama_index.core import load_index_from_storage
storage_context = StorageContext.from_defaults(graph_store=graph_store, persist_dir="graph_store")
# Try to load the index, if it doesn't exist, create it

try:
    kg_index = load_index_from_storage(storage_context)
    print("Loaded existing index")
except ValueError:
    print("Creating new index")
    # Assuming 'nodes' is your list of document nodes
    kg_index = PropertyGraphIndex(
        nodes,
        max_triplets_per_chunk=2,
        show_progress=True,
        storage_context=storage_context
    )
    persist_dir = "graph_store"
    kg_index.storage_context.persist(persist_dir=persist_dir)

In [ ]:
kg_index.storage_context.persist(persist_dir=persist_dir)

In [ ]:
kg_index.as_query_engine("What is Wollastonite?")


In [ ]:
   query_engine = kg_index.as_query_engine()
   response = query_engine.query("What are the industrial applications of Wollastonite?")
   print(response)

In [ ]:
# Load existing knowledge graph.
from llama_index.graph_stores.neo4j import Neo4jPropertyGraphStore
from llama_index.core import PropertyGraphIndex
from llama_index.core.storage import StorageContext

neo4j_uri = "bolt://localhost:7687"  # Update with your Neo4j instance details
neo4j_user = "neo4j"
neo4j_password = "asd@123qwe"

graph_store = Neo4jPropertyGraphStore(
    username=neo4j_user,
    password=neo4j_password,
    url=neo4j_uri,
    database="test",  # Use appropriate database name
)
# Create a StorageContext with your graph_store
storage_context = StorageContext.from_defaults(property_graph_store=graph_store)

kg_index = PropertyGraphIndex.from_existing(
    property_graph_store=graph_store,
    storage_context=storage_context,
    llm=Settings.llm,
    embed_model=Settings.embed_model,
)


In [90]:
from llama_index.core import PropertyGraphIndex

# create
index = PropertyGraphIndex.from_documents(
    loaded_documents,
)

# use
retriever = index.as_retriever(
    include_text=True,  # include source chunk with matching paths
    similarity_top_k=2,  # top k for vector kg node retrieval
)
nodes = retriever.retrieve("<QUERY>")

query_engine = index.as_query_engine(
    include_text=True,  # include source chunk with matching paths
    similarity_top_k=2,  # top k for vector kg node retrieval
)
response = query_engine.query("what is wollastonite?")

# save and load
index.storage_context.persist(persist_dir="./storage")

from llama_index.core import StorageContext, load_index_from_storage

index = load_index_from_storage(
    StorageContext.from_defaults(persist_dir="./storage")
)

# loading from existing graph store (and optional vector store)
# load from existing graph/vector store
index = PropertyGraphIndex.from_existing(
    property_graph_store=graph_store, vector_store=vector_store, llm=llm
)

UnicodeEncodeError: 'charmap' codec can't encode character '\u2192' in position 17032: character maps to <undefined>

In [94]:
# save and load
index.storage_context.persist(persist_dir="graph_store")

TypeError: Object of type set is not JSON serializable

In [93]:
import json
from typing import Any
from llama_index.core.graph_stores.simple_labelled import SimplePropertyGraphStore

def custom_persist(self, persist_path: str, fs: Any = None) -> None:
    if fs is None:
        import fsspec
        fs = fsspec.filesystem("file")
    with fs.open(persist_path, "w", encoding='utf-8') as f:
        json.dump(self.graph.model_dump(), f, ensure_ascii=False)

# Monkey-patch the persist method
SimplePropertyGraphStore.persist = custom_persist

In [ ]:
kg_index

In [ ]:
def inspect_property_graph_index(kg_index):
    print("Inspecting PropertyGraphIndex\n")

    # 1. Check the property graph store
    print(f"Property graph store type: {type(kg_index.property_graph_store)}")

    # 2. Get all nodes
    all_nodes = kg_index.property_graph_store.get()
    print(f"\nNumber of nodes: {len(all_nodes)}")
    print("Sample nodes:", all_nodes[:5] if len(all_nodes) > 5 else all_nodes)

    # 3. Inspect a specific node
    if all_nodes:
        sample_node = all_nodes[0]
        print(f"\nSample node:")
        print(f"  ID: {sample_node.id}")
        print(f"  Labels: {sample_node.labels}")
        print(f"  Properties: {sample_node.properties}")

        # 4. Get relationships for the sample node
        relationships = kg_index.property_graph_store.get_node_relationships(sample_node.id)
        print(f"\nRelationships for sample node:")
        for rel in relationships:
            print(f"  {rel['source']} -{rel['relation']}-> {rel['target']}")

    # 5. Perform a query using the index's retriever
    try:
        retriever = kg_index.as_retriever()
        query = "What is Wollastonite?"
        results = retriever.retrieve(query)

        print(f"\nQuery: {query}")
        print("Results:")
        for i, result in enumerate(results, 1):
            print(f"\nResult {i}:")
            print(f"Node ID: {result.node.node_id}")
            print(f"Score: {result.score}")
            print(f"Content: {result.node.get_content()[:100]}...")  # First 100 characters
    except Exception as e:
        print(f"Error during retrieval: {str(e)}")

    # 6. Check schema
    if kg_index.property_graph_store.supports_structured_queries:
        schema = kg_index.property_graph_store.get_schema()
        print("\nGraph Schema:")
        print(schema)
    else:
        print("\nStructured queries not supported by this graph store.")

# Use the function
inspect_property_graph_index(kg_index)

In [ ]:
# Add these imports if not already present
from neo4j import GraphDatabase

# Use the same connection details you used for Neo4jPropertyGraphStore
driver = GraphDatabase.driver(neo4j_uri, auth=(neo4j_user, neo4j_password))

def run_query(query):
    with driver.session(database="test") as session:
        result = session.run(query)
        return [record for record in result]

# Count all nodes
node_count = run_query("MATCH (n) RETURN count(n) as count")
print(f"Total nodes: {node_count[0]['count']}")

# Count all relationships
rel_count = run_query("MATCH ()-->() RETURN count(*) as count")
print(f"Total relationships: {rel_count[0]['count']}")

# Get node labels
labels = run_query("CALL db.labels()")
print("Node labels:", [label[0] for label in labels])

# Get relationship types
rel_types = run_query("CALL db.relationshipTypes()")
print("Relationship types:", [rel[0] for rel in rel_types])

# Sample of nodes (adjust LIMIT as needed)
sample_nodes = run_query("MATCH (n) RETURN n LIMIT 5")
print("Sample nodes:")
for node in sample_nodes:
    print(dict(node['n']))

driver.close()

In [ ]:
   print("Creating PropertyGraphIndex...")
   kg_index = PropertyGraphIndex.from_existing(
       property_graph_store=graph_store,
       llm=Settings.llm,
       embed_model=Settings.embed_model,
   )
   print("PropertyGraphIndex created.")

   # Try to access the underlying graph store
   if hasattr(kg_index, 'property_graph_store'):
       print("Graph store accessible from kg_index")
       # Try to get nodes from the graph store
       try:
           nodes = kg_index.property_graph_store.get()
           print(f"Nodes retrieved from kg_index.graph_store: {len(nodes)}")
       except Exception as e:
           print(f"Error retrieving nodes from kg_index.graph_store: {str(e)}")
   else:
       print("No direct access to graph_store from kg_index")

   # Try to perform a query
   query_engine = kg_index.as_query_engine()
   response = query_engine.query("What is Wollastonite?")
   print("Query response:", response)

In [ ]:
from llama_index.retrievers.bm25 import BM25Retriever

bm25_retriever = BM25Retriever(
    nodes=nodes,  # These are the same nodes you used for other indexes
    similarity_top_k=5,
    verbose=True
)
bm25_retriever.persist("bm25_index")

In [ ]:
from llama_index.retrievers.bm25 import BM25Retriever
bm25_retriever = BM25Retriever.from_persist_dir("bm25_index")

# Set up the Retriever
Nodes of text will be retrieved from three different spaces:
- a vector index based on semantic similarity.
- a bm25 index based on keyword matching.
- a knowledge graph index based on entities and their relationships.
Duplicates are removed and then a Cohere rerank is applied.

In [ ]:
# Build Retriever
from src.hybrid_graph_retriever import HybridGraphRetriever
retriever = HybridGraphRetriever(
    vector_index=vector_index,
    kg_index=kg_index,
    bm25_retriever=bm25_retriever,
)

# Run the Query and Count Tokens
We can finally ask our question!

In [ ]:
from llama_index.core import query_engine
from llama_index.llms.ollama import Ollama
from llama_index.core import PromptTemplate

# Assuming you've already set up your retriever as shown:
# retriever = HybridGraphRetriever(
#     vector_index=vector_index,
#     kg_index=kg_index,
#     bm25_retriever=bm25_retriever,
# )

# Set up Ollama LLM
llm = Ollama(model="mistral")

# Create a custom prompt template
custom_prompt_template = PromptTemplate(
    "You are an AI assistant specializing in soil analysis and plant nutrition. "
    "Using only the information provided in the context, answer the question. "
    "If you cannot answer based solely on the given context, say 'I don't have enough information to answer that question.'\n"
    "Context: {context_str}\n"
    "Question: {query_str}\n"
    "Answer: "
)



# Example usage:
# response = query_engine.query("What is the optimal pH for cannabis plants?")
# print(response)

In [ ]:
from llama_index.llms.ollama import Ollama
from llama_index.core.schema import QueryBundle
from llama_index.core import PromptTemplate

# Initialize the Ollama LLM
# We are directly using the Ollama class in order to get to the tokens.
ollama_llm = Ollama(model="mistral")

# Create a custom prompt template for Ollama
custom_prompt_template = PromptTemplate(
    "You are an AI assistant specializing in soil analysis and plant nutrition. "
    "Using only the information provided in the context, answer the question. "
    "If you cannot answer based solely on the given context, say 'I don't have enough information to answer that question.'\n"
    "Context: {context_str}\n"
    "Question: {query_str}\n"
    "Answer: "
)

# Function to run a query with the retriever and custom prompt
def ask_question(query: str):
    # Use the retriever to get relevant context
    query_bundle = QueryBundle(query_str=query)
    retrieved_nodes = retriever.retrieve(query_bundle)

    # Prepare the context string from retrieved nodes
    context_str = "\n".join([node.node.text for node in retrieved_nodes])

    # Format the prompt using the custom template
    formatted_prompt = custom_prompt_template.format(
        context_str=context_str,
        query_str=query
    )

    # Use Ollama's chat method with the formatted prompt
    from llama_index.core.base.llms.types import ChatMessage, MessageRole

    messages = [ChatMessage(role=MessageRole.USER, content=formatted_prompt)]
    ollama_response = ollama_llm.chat(messages)

    return {
        "query": query,
        "answer": ollama_response.message.content,
        "contexts": [node.node.text for node in retrieved_nodes],
        "token_info": {
            "prompt_tokens": ollama_response.raw.get('prompt_eval_count', 0),
            "completion_tokens": ollama_response.raw.get('eval_count', 0),
            "total_tokens": ollama_response.raw.get('prompt_eval_count', 0) + ollama_response.raw.get('eval_count', 0)
        },
        "other_info": {
            "model": ollama_response.raw.get('model'),
            "total_duration": ollama_response.raw.get('total_duration'),
            "load_duration": ollama_response.raw.get('load_duration'),
            "eval_duration": ollama_response.raw.get('eval_duration')
        }
    }

In [ ]:

import re
# --->: Set up the local embedding model and LLM
# Set embedding model



class SimpleFaithfulness:
    def __init__(self):
        pass

    def evaluate(self, question, answer, context):
        prompt = f"""
        Question: {question}
        Answer: {answer}
        Context: {context}

        Evaluate the faithfulness of the answer based on the given context. Consider the following:
        1. Does the answer contain information not present in the context?
        2. Does the answer contradict any information in the context?
        3. Is the answer a fair representation of the information in the context?

        Respond with:
        1. A score from 0 to 1, where 0 is completely unfaithful and 1 is completely faithful.
        2. A brief explanation of your scoring.
        3. Any hallucinations or discrepancies found, if any.

        Format your response exactly as follows:
        Score: [Your score here]
        Explanation: [Your explanation here]
        Hallucinations: [List any hallucinations or discrepancies, or 'None' if none found]
        """

        try:
            response = ask_question(prompt)
            return self._parse_response(response)
        except Exception as e:
            print(f"An error occurred during evaluation: {str(e)}")
            return {"faithfulness": 0, "explanation": "Error occurred", "hallucinations": "Unable to evaluate"}

    def _parse_response(self, response):
        score_match = re.search(r'Score:\s*([\d.]+)', response['answer'])
        explanation_match = re.search(r'Explanation:\s*(.+?)(?:\n|$)', response['answer'], re.DOTALL)
        hallucinations_match = re.search(r'Hallucinations:\s*(.+?)(?:\n|$)', response['answer'], re.DOTALL)

        score = float(score_match.group(1)) if score_match else 0
        explanation = explanation_match.group(1).strip() if explanation_match else "No explanation provided"
        hallucinations = hallucinations_match.group(1).strip() if hallucinations_match else "Unable to determine"

        return {
            "faithfulness": score,
            "explanation": explanation,
            "hallucinations": hallucinations
        }

# Usage example
simple_faithfulness = SimpleFaithfulness()
# Example usage of simple_faithfulness.evaluate()
question = "What are the benefits of using wollastonite in agriculture?"
# answer = "Wollastonite is beneficial in agriculture due to its liming capability, silicon content, and calcium content. It can help improve soil pH and provide essential nutrients to plants."
answer = "The sky is purple."
context = "Wollastonite is a calcium silicate mineral. It is used in agriculture for its liming capability, silicon content, and calcium content. These properties can help improve soil structure and provide nutrients to plants."

result = simple_faithfulness.evaluate(question, answer, context)
print(f"Faithfulness score: {result['faithfulness']}")
print(f"Explanation: {result['explanation']}")
print(f"Hallucinations: {result['hallucinations']}")


In [ ]:
from ragas.metrics import (
    faithfulness,
    answer_relevancy,
    context_precision,
    context_recall
)

In [ ]:
from langchain_ollama.llms import OllamaLLM
from ragas.metrics import Faithfulness
from datasets import Dataset

# Create an instance of OllamaLLM
ollama_llm = OllamaLLM(model="mistral")

# Create an instance of the Faithfulness metric with your LLM
custom_faithfulness = Faithfulness(llm=ollama_llm)

def evaluate_single_response(question, answer, contexts):
    dataset = Dataset.from_dict({
        "user_input": question,
        "response": answer,
        "context": contexts,
        "ground_truths": [""]  # Empty string instead of empty list
    })

    # Use the custom faithfulness metric
    result = custom_faithfulness.score(dataset)

    return result['faithfulness'][0]

# Example usage
question = "What is wollastonite?"
answer = "Wollastonite is a calcium silicate mineral used in agriculture for its liming capability and silicon content."
contexts = "Wollastonite is a calcium silicate mineral. It is used in agriculture for its liming capability, silicon content, and calcium content."

faithfulness_score = evaluate_single_response(question, answer, contexts)
print(f"Faithfulness score: {faithfulness_score}")

In [ ]:
evaluation_data = [
    {
        "question": "What is wollastonite and how does it relate to plant nutrition?",
        "ground_truth": "Wollastonite is a calcium silicate mineral used in agriculture for its liming capability, silicon content, and calcium content. It relates to plant nutrition by providing calcium and silicon, buffering soil pH, and potentially contributing to pest control and powdery mildew suppression."
    },
    # Add more question-answer pairs here
]